<a href="https://colab.research.google.com/github/iskanor1/handwritten-arabic-/blob/main/Handwritten%20Arabic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
# -*- coding: utf-8 -*-
"""Welcome To Colab

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/notebooks/intro.ipynb
"""

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# ----------------------------------------------------------------------
# Configuration
# ----------------------------------------------------------------------
IMAGE_DATA_PATH_TRAIN = '/content/csvTrainImages 13440x1024.csv'
LABEL_DATA_PATH_TRAIN = '/content/csvTrainLabel 13440x1.csv'
IMAGE_DATA_PATH_TEST = '/content/csvTestImages 3360x1024.csv'
LABEL_DATA_PATH_TEST = '/content/csvTestLabel 3360x1.csv'

TF_EPOCHS = 20  # Adjusted epochs for potentially faster convergence
TF_BATCH_SIZE = 64
RANDOM_SEED = 42
VALIDATION_SIZE = 0.2  # Fraction of training data to use for validation
IMAGE_WIDTH = 32
IMAGE_HEIGHT = 32
NUM_CHANNELS = 1

# Set random seeds for reproducibility
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

# ----------------------------------------------------------------------
# 1. Load Data from CSV
# ----------------------------------------------------------------------

def load_data_from_csv(file_path, is_label=False):
    """Loads data from a CSV file.

    Args:
        file_path (str): Path to the CSV file.
        is_label (bool): If True, reshapes the output to a 1D array for labels.

    Returns:
        numpy.ndarray: Data as a NumPy array.
    """
    try:
        df = pd.read_csv(file_path, header=None)
        data = df.values.astype('float32')
        if not is_label:
            data = data.reshape(-1, IMAGE_HEIGHT, IMAGE_WIDTH, NUM_CHANNELS) / 255.0  # Normalize and reshape for CNN
        else:
            data = data.flatten()
        return data
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        return None

# Load the data
X_train_full = load_data_from_csv(IMAGE_DATA_PATH_TRAIN)
y_train_full = load_data_from_csv(LABEL_DATA_PATH_TRAIN, is_label=True)
X_test = load_data_from_csv(IMAGE_DATA_PATH_TEST)
y_test = load_data_from_csv(LABEL_DATA_PATH_TEST, is_label=True)

if X_train_full is None or y_train_full is None or X_test is None or y_test is None:
    exit()

# Split training data into training and validation sets for TensorFlow
X_train, X_val, y_train, y_val = train_test_split(
    X_train_full, y_train_full, test_size=VALIDATION_SIZE, random_state=RANDOM_SEED
)

# Determine the number of classes
num_classes = int(np.max(y_train_full)) + 1
print(f"Number of classes: {num_classes}")

# ----------------------------------------------------------------------
# 2. TensorFlow/Keras Implementation (CNN)
# ----------------------------------------------------------------------
def create_tf_cnn_model(input_shape, num_classes):
    """Creates a TensorFlow/Keras CNN model.

    Args:
        input_shape (tuple): Shape of the input images (height, width, channels).
        num_classes (int): Number of classes.

    Returns:
        tensorflow.keras.models.Sequential: A TensorFlow/Keras CNN model.
    """
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape, name='conv2d_1'),
        layers.MaxPooling2D((2, 2), name='max_pooling2d_1'),
        layers.Conv2D(64, (3, 3), activation='relu', name='conv2d_2'),
        layers.MaxPooling2D((2, 2), name='max_pooling2d_2'),
        layers.Flatten(name='flatten'),
        layers.Dense(128, activation='relu', name='dense_1'),
        layers.Dropout(0.5, name='dropout_1'),
        layers.Dense(num_classes, activation='softmax', name='output')
    ], name="tf_cnn_model")
    return model

def train_tf_cnn_model(model, X_train, y_train, X_val, y_val, num_classes, epochs=TF_EPOCHS, batch_size=TF_BATCH_SIZE):
    """Trains a TensorFlow/Keras CNN model.

    Args:
        model (tensorflow.keras.models.Sequential): The model to train.
        X_train (numpy.ndarray): Training image data.
        y_train (numpy.ndarray): Training labels.
        X_val (numpy.ndarray): Validation image data.
        y_val (numpy.ndarray): Validation labels.
        num_classes (int): Number of classes.
        epochs (int): Number of training epochs (default: TF_EPOCHS).
        batch_size (int): Batch size (default: TF_BATCH_SIZE).

    Returns:
        tensorflow.keras.callbacks.History: Training history.
    """
    # One-hot encode the labels for tensorflow
    y_train_encoded = to_categorical(y_train, num_classes=num_classes)
    y_val_encoded = to_categorical(y_val, num_classes=num_classes)
    y_test_encoded = to_categorical(y_test, num_classes=num_classes) # Encode test labels as well

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    model.summary()
    history = model.fit(X_train, y_train_encoded, epochs=epochs, batch_size=batch_size,
                        validation_data=(X_val, y_val_encoded), verbose=1)
    return history

def evaluate_tf_cnn_model(model, X_test, y_test, num_classes):
    """Evaluates a TensorFlow/Keras CNN model.

    Args:
        model (tensorflow.keras.models.Sequential): The model to evaluate.
        X_test (numpy.ndarray): Test image data.
        y_test (numpy.ndarray): Test labels.
        num_classes (int): Number of classes.

    Returns:
        float: Test accuracy.
    """
    y_test_encoded = to_categorical(y_test, num_classes=num_classes)
    loss, accuracy = model.evaluate(X_test, y_test_encoded, verbose=0)
    print(f'TensorFlow/Keras CNN Test Accuracy: {accuracy:.4f}')
    return accuracy

if __name__ == '__main__':
    input_shape_tf = (IMAGE_HEIGHT, IMAGE_WIDTH, NUM_CHANNELS)

    # ----------------------------------------------------------------------
    # TensorFlow/Keras (CNN)
    # ----------------------------------------------------------------------
    tf_cnn_model = create_tf_cnn_model(input_shape_tf, num_classes)
    tf_cnn_history = train_tf_cnn_model(tf_cnn_model, X_train, y_train, X_val, y_val, num_classes)
    tf_cnn_accuracy = evaluate_tf_cnn_model(tf_cnn_model, X_test, y_test, num_classes)

Number of classes: 29


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "tf_cnn_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_1 (Conv2D)               │ (None, 30, 30, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 13, 13, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2304)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │       295,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 29)             │         3,741 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 317,597 (1.21 MB)

 Trainable params: 317,597 (1.21 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
168/168 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.1508 - loss: 2.9046 - val_accuracy: 0.5733 - val_loss: 1.4344
Epoch 2/20
168/168 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.4880 - loss: 1.5877 - val_accuracy: 0.7277 - val_loss: 0.8923
Epoch 3/20
168/168 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6360 - loss: 1.0919 - val_accuracy: 0.8155 - val_loss: 0.6300
Epoch 4/20
168/168 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7186 - loss: 0.8492 - val_accuracy: 0.8270 - val_loss: 0.5514
Epoch 5/20
168/168 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7598 - loss: 0.7207 - val_accuracy: 0.8609 - val_loss: 0.4460
Epoch 6/20
168/168 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8032 - loss: 0.6108 - val_accuracy: 0.8817 - val_loss: 0.3797
Epoch 7/20
168/168 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8248 - loss: 0.5404 - val_accuracy: 0.8888 - val_loss: 0.3426
Epoch 8/20
168/168 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8390 - loss: 0.4771 - val_accuracy: 0.

In [16]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image

def preprocess_image_for_model(img_path, target_size=(32, 32)):
    """
    Preprocesses a single image for use with a TensorFlow model that expects
    flattened input.  This includes loading, resizing, normalizing, and flattening.

    Args:
        img_path (str): Path to the image file.
        target_size (tuple, optional): The size to resize the image to (height, width).
            Defaults to (32, 32).  This should match the size of images
            your model was trained on.

    Returns:
        numpy.ndarray: A flattened NumPy array representing the preprocessed image,
                         or None if there's an error.
    """
    try:
        # 1. Load the image
        img = image.load_img(img_path, target_size=target_size)

        # 2. Convert to a NumPy array
        img_array = image.img_to_array(img)

        # 3. Expand dimensions to create a batch of size 1
        img_array = np.expand_dims(img_array, axis=0)

        # 4. Normalize the pixel values (important for most models)
        img_array = img_array / 255.0

        # 5. Flatten the image
        flattened_img = img_array.flatten()

        return flattened_img

    except FileNotFoundError:
        print(f"Error: Image not found at {img_path}")
        return None
    except Exception as e:
        print(f"An error occurred while processing the image: {e}")
        return None

def test_model_with_image(model, img_path, target_size=(32, 32)):
    """
    Loads an image, preprocesses it, and uses a TensorFlow model to make a prediction.

    Args:
        model (tensorflow.keras.Model): Your trained TensorFlow model.
        img_path (str): Path to the image file.
        target_size (tuple, optional): The size to resize the image to.
            Defaults to (32, 32).
    """
    flattened_image = preprocess_image_for_model(img_path, target_size)
    if flattened_image is None:
        print("Error: Image preprocessing failed.")
        return

    # важно!!! Reshape the flattened image to match the input shape the model expects.
    # The model was trained on flattened images of a certain dimension.
    input_shape = model.input_shape
    # Remove the batch dimension (None) and get the expected number of features
    expected_features = input_shape[1]  # Get size after flattening
    reshaped_image = flattened_image.reshape(1, expected_features) # Add batch dimension

    # 6. Make a prediction
    predictions = model.predict(reshaped_image)

    # Process the predictions (this will depend on your model's output)
    # For example, if it's a classification model:
    class_index = np.argmax(predictions[0])  # Get the index of the highest probability
    print(f"Raw Predictions: {predictions}")
    print(f"Predicted class index: {class_index}")
    return predictions # Return the raw predictions

def main():
    """
    Main function to demonstrate the image preprocessing and testing process.
    """
    # 1. Load your trained TensorFlow model
    # Replace 'your_model.h5' with the actual path to your saved model file
    try:
        model = tf.keras.models.load_model('your_model.h5')
    except Exception as e:
        print(f"Error: Failed to load model: {e}")
        return

    # 2. Specify the path to the image you want to test
    image_path = '/content/Untitled.png'  # Replace with a valid image path

    # 3. Call the function to test the image
    predictions = test_model_with_image(model, image_path)
    if predictions is not None:
       print("Successfully made predictions")

if __name__ == "__main__":
    main()


Error: Failed to load model: [Errno 2] Unable to synchronously open file (unable to open file: name = 'your_model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)
